<a href="https://colab.research.google.com/github/AndyCoder2023/CE4145-NLP-Coursework/blob/main/CE4145_Coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

“Generative AI was used to support completion of this assessment. If used: The GenAI tool [Gemini] was used for the purpose of [ideation/editing]. Where used for the purpose of supporting development,
comments have been provided against relevant cells.

Prompt 1:
Please explain this error:
The error FileNotFoundError: hf://datasets/Pablinho/movies-dataset/9000plus.csv not found in cell 8Ypu2LZD3FOP means that the code is looking for a file at the specified Hugging Face dataset path, but it cannot find it.
Looking at the available variables, it seems the movie data is already loaded into the movie_df DataFrame. I will remove the redundant data loading code from cell 8Ypu2LZD3FOP and use the existing movie_df to create the x and y variables for the NLP pipeline.
It seems there are missing values in the data that are causing the ValueError: Input contains NaN. I will add code to remove rows with missing values in the 'Overview' or 'Genre' columns before processing the data.


**Describe the dataset and the task to be performed**

# most occuring genre
# list all the superheroes

# Dataset

The dataset has 9 columns with 9.84k rows of movie data. The task that is to be performed for the dataset will consist of two different pipelines which will compare the best strategy for the given task to be performed on the movie dataset. The main problem solving task at hand will to find out what the  superhero names are using the statistics available in the dataset. An NLP system is required for this objective to find and narrow down the dataset to pin point the superheros that occur in the dataset. By implementing an NLP system there can be strong analysis on the dataset to find the correct criteria.

** Debating whether to use this: Merchán-Rivera, P. (2025). movies-dataset [Dataset]. Hugging Face. https://huggingface.co/datasets/Pablinho/movies-dataset.

Reference: https://colab.research.google.com/drive/1MbiTzuPPCKrj9BOu_Nj4gcsxxYKwLg0a?usp=sharing#scrollTo=rubk2y4rGx7y


In [41]:
import io #io we use to load the data after uploading (as colab uses dictionaries to store uploaded files)
from google.colab import files #finally, we import the files package from google.colab framework to be able to upload files

from sklearn.pipeline import Pipeline #let's import the pipeline functionality
from sklearn.feature_extraction.text import CountVectorizer #and we will import a simple pre-processing method
from sklearn.feature_extraction.text import TfidfTransformer #and a representation learner
from sklearn.neighbors import KNeighborsClassifier #and a simple classifier model
from sklearn.model_selection import StratifiedKFold #cross fold is sometimes called k-fold. Calling the stratified version ensures that classes have equal representation across folds
from sklearn.metrics import accuracy_score #import an accuracy metric to tell us how well the model is doing

import numpy as np
np.random.seed(1337)

import pandas as pd

# Used ai to fix gathering dataset not being found.

# Drop rows with missing values in 'Overview' or 'Genre' columns
movie_df.dropna(subset=['Overview', 'Genre'], inplace=True)

x = movie_df['Overview'].values
y = movie_df['Genre'].values

# dataset = np.loadtxt(sentiment_file, dtype=str, delimiter="\t") #we will convert to a numpy array, where the first column is the text and the second column is the label
# print(dataset.shape) #if we print the shape, we can see we have 1000 examples, each of which is associated with label
# x, y = dataset.T #A quick way to do this is to transpose the dataset and split into two variables

text_clf = Pipeline([ #the pipeline object allows us to organise a series of functions which will be applied to our text data as though they were a single function
  ('prep', CountVectorizer()), #we will use a simple count vectorizer for our pre-processing (which cheats a little by combining numerous pre-processing steps)
  ('rep', TfidfTransformer()), #and a representation learning method using tf-idf
  ('mod', KNeighborsClassifier()), #and a simple kNN classifier
  ])

acc_score = [] #create a list to store the accuracy values

kf = StratifiedKFold(n_splits=5) #we instantiate the kfold instance, and set the number of folds to 5
for train, test in kf.split(x,y): #we use a for loop to iterate through each fold using the train and test indexes from the dataset

  x_train, x_test, y_train, y_test = x[train], x[test], y[train], y[test] #things can get a bit weird when inputting indexes to functions, so lets save them as variables
  #print(train)
  #print(test) #this will print the train and test indexes respectively, if you want to be sure they do not overlap

  text_clf.fit(x_train, y_train) #we then only fit the training data (note that we oapply the text_clf pipeline object, rather than having to go through each function separately)
  predictions = text_clf.predict(x_test) #and can predict on the test data (similar to above, we can predict using the pipeline directly)
  acc = accuracy_score(predictions, y_test) #we use the accuracy score we imported to give an idea how well the model is doing
  acc_score.append(acc) #we can append it to our list

print("Accuracy:", np.mean(acc_score))

/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Accuracy: 0.05119199418097479


In [43]:
import nltk #import the natural language toolkit

nltk.download('punkt') #download the package in nltk which supports tokenization
nltk.download('stopwords') #download the nltk package for stopwords

from nltk.tokenize import word_tokenize #import the tokenize package
from nltk.corpus import stopwords #import the package from the corpus
from nltk.stem.snowball import SnowballStemmer #import the snowball stemmer (also known as Porter2)

from sklearn.base import BaseEstimator, TransformerMixin

class pre_process(BaseEstimator, TransformerMixin):

    def __init__(self):
      return None #we do not need any parameters to instantiate this class

    def fit(self, X, y=None): #both fit and transform expect the data instances and labels to be called - we do not use the labels, so set y=None
        return self #as explained above, we will not use the fit method

    def transform(self, X, y=None):
      prep_text = []
      for x in X: #for each sentence in the whole dataset
            token_text = word_tokenize(x) #tokenize the document
            normd_text = [token.lower() for token in token_text if token.isalpha()] #list compression to apply some simple cleaning (lower case casting and punctuation removal) to tokenized terms

            swr_text = [token for token in normd_text if token not in stopwords.words('english')] #list compression to remove any stopwords from our list

            stemmer = SnowballStemmer("english") #specify we are using the English stemming rules, as other languages are present in toolkit
            prep_text += [[stemmer.stem(word) for word in swr_text]] #list compression for applying the stemmer

      prep_sentences = [" ".join(sentence) for sentence in prep_text] #we join the sentences back together to ensure compatibility with CountVec, which is doing some of it's own prep
      return prep_sentences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Describe the representation learning
approach adopted. **bold text**

# Representation Learning

The reprensentation learning method used will be the Word-2-Vector(Word2Vec). Word2Vec is a technique which is in Natural language processing. The technique permits words to have reprentation as vectors which are in a continuous vector space. The representation learned for every word will be "prediciting a target term using its context(CBOW)"(Martin, 2025, slide 49). Continous Bag of Words (Continuos Bag of Words) model will be used to predict a word that is given words that is located in a window. Input layer will have the context words while output layer will have a current word. Hidden layer has dimensions that will show a current word that is located on the output layer.(Kadam, 2025)


Kyle, M.(2025)'Natural Language Processing'[PowerPoint presentation]. CE4145: CE4145 & CEM300 W5 - Text Pre-Processing & Representation Learning. Available at: https://campusmoodle.rgu.ac.uk/mod/resource/view.php?id=5411825 (Accessed: 18 October 2025).

sumedh, K.(2025) Word Embedding using Word2Vec. Available at: https://www.geeksforgeeks.org/python/python-word-embedding-using-word2vec/ (Accessed: 22 October 2025).

In [5]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 58.4 MB/s eta 0:00:00


In [44]:
from gensim.models import Word2Vec #first we import word2vec, then a few familiar imports

In [46]:
import gensim
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt_tab')
import warnings

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [50]:
data = []

for i in sent_tokenize(movie_df):
    temp = []

    # tokenize the sentence into words
    for j in word_tokenize(i):
        temp.append(j.lower())

    data.append(temp)

TypeError: expected string or bytes-like object, got 'DataFrame'

In [48]:
model1 = gensim.models.Word2Vec(data, min_count=1,
                                vector_size=100, window=5)

RuntimeError: you must first build vocabulary before training the model

Describe the theory behind the
algorithms to be applied. **bold text**

# Algorithms

The two algorithms that will implemented will consist of Convolutional Neural Network and Siamese Neural Network. Convolutional Neural Network(CNN) will have a dense layer which has the convolution with max pooling layers. Will we able to summarise the images of the movies that appear the most to gather what movie is most popular by the number of images appearing. In the convolution layer of the CNN there will be neurons that can detect small parts of the image to determine what the image is showing. For the max pooling layer this will be at the output of the convolution layer. A pooling operation will be used to summarise a feature map. Subsampling will be used to make the images smaller to give the network lesser parameters for better computation. The cnn will work by a series of steps which are as follows: Convolution, max pooling, convolution, max pooling, and then the final step which is flatten.

Siamese Neural Network will have identical weights that are shared by two neural networks which are also joined at one or possiblly more than one layer. The cases have positive or negative labels and are in pairs. This network will learn representation where connecting pair members are right next to each other while the pair members which are non-matching will exist margin apart

Describe the evaluation process, and
analyse the results. **bold text**

# Evaluation